In [ ]:
"""
MatrixTransforms_Manim.py

Manim v0.19.0-ready scene to visualize 2x2 matrix multiplication and geometric transformations.

Features:
 - Show basis vectors and unit square in a NumberPlane.
 - Apply matrix B, then A (visualizing composition A \u2218 B).
 - Compute C = A @ B and show numeric equality: A(B(v)) == C(v).
 - Animate entry-wise matrix multiplication (dot-product) to produce C's entries.
 - Show eigenvectors (if real) of A and how A scales them by eigenvalues.

Run with:
    manim -pql MatrixTransforms_Manim.py MatrixTransformScene
"""

"""
MatrixTransforms_Manim_v019.py

Fully compatible with Manim Community v0.19.0

Features:
 - Basis vectors and unit square.
 - Apply matrices B then A (composition).
 - Show C = AB.
 - Animate entry-wise matrix multiplication.
 - Eigenvectors visualization.
 - Manual coordinate labels.

Run with:
    manim -pql MatrixTransforms_Manim_v019.py MatrixTransformScene
"""

from manim import *
import numpy as np

class MatrixTransformScene(Scene):
    def construct(self):
        # === Configuration ===
        A = np.array([[1.2, -0.6], [0.8, 1.1]])
        B = np.array([[0.5, 0.3], [-0.2, 1.4]])
        C = A @ B

        # === Setup NumberPlane ===
        plane = NumberPlane(
            x_range=[-6,6,1],
            y_range=[-4,4,1],
            background_line_style={"stroke_color": GREY_A, "stroke_opacity": 0.4}
        )
        self.add(plane)

        # Manual coordinate labels
        x_labels = VGroup(*[MathTex(str(x)).next_to(plane.c2p(x,0), DOWN) for x in range(-6,7)])
        y_labels = VGroup(*[MathTex(str(y)).next_to(plane.c2p(0,y), LEFT) for y in range(-4,5)])
        coord_labels = VGroup(x_labels, y_labels)
        self.add(coord_labels)

        # === Basis vectors ===
        e1 = Arrow(plane.c2p(0,0), plane.c2p(1,0), buff=0).set_color(BLUE)
        e2 = Arrow(plane.c2p(0,0), plane.c2p(0,1), buff=0).set_color(GREEN)
        e1_label = MathTex("\\mathbf{e}_1").next_to(e1.get_end(), RIGHT, buff=0.05)
        e2_label = MathTex("\\mathbf{e}_2").next_to(e2.get_end(), UP, buff=0.05)

        # Unit square
        square = Polygon(
            plane.c2p(0,0), plane.c2p(1,0), plane.c2p(1,1), plane.c2p(0,1),
            color=YELLOW, fill_opacity=0.25
        )
        square_label = MathTex("S").next_to(square.get_center(), UP)

        # Original copies for C transformation
        e1_orig = e1.copy()
        e2_orig = e2.copy()
        square_orig = square.copy()

        self.play(GrowArrow(e1), GrowArrow(e2), Write(e1_label), Write(e2_label))
        self.play(Create(square), Write(square_label))
        self.wait(0.6)

        # === Show matrices ===
        matA = Matrix([[f"{A[0,0]:.2f}", f"{A[0,1]:.2f}"],[f"{A[1,0]:.2f}", f"{A[1,1]:.2f}"]]).to_corner(UL)
        matA_label = MathTex("A").next_to(matA, LEFT)
        matB = Matrix([[f"{B[0,0]:.2f}", f"{B[0,1]:.2f}"],[f"{B[1,0]:.2f}", f"{B[1,1]:.2f}"]]).to_corner(UR)
        matB_label = MathTex("B").next_to(matB, RIGHT)
        self.play(Write(matA), Write(matA_label), Write(matB), Write(matB_label))
        self.wait(0.6)

        # === Apply B ===
        e1_B = e1.copy(); e1_B.apply_matrix(B)
        e2_B = e2.copy(); e2_B.apply_matrix(B)
        square_B = square.copy(); square_B.apply_matrix(B)
        self.play(Transform(e1, e1_B), Transform(e2, e2_B), Transform(square, square_B))
        self.play(Indicate(matB, scale_factor=1.1))
        self.wait(0.6)

        # === Apply A after B (composition) ===
        e1_BA = e1.copy(); e1_BA.apply_matrix(A)
        e2_BA = e2.copy(); e2_BA.apply_matrix(A)
        square_BA = square.copy(); square_BA.apply_matrix(A)
        self.play(Transform(e1, e1_BA), Transform(e2, e2_BA), Transform(square, square_BA))
        self.play(Indicate(matA, scale_factor=1.1))
        self.wait(0.6)

        # === Show C = AB ===
        matC = Matrix([[f"{C[0,0]:.2f}", f"{C[0,1]:.2f}"],[f"{C[1,0]:.2f}", f"{C[1,1]:.2f}"]]).next_to(matA, DOWN, buff=0.8)
        matC_label = MathTex("C = AB").next_to(matC, LEFT)
        self.play(Write(matC), Write(matC_label))
        self.wait(0.6)

        # === Apply C directly to original objects ===
        e1_C = e1_orig.copy(); e1_C.apply_matrix(C)
        e2_C = e2_orig.copy(); e2_C.apply_matrix(C)
        square_C = square_orig.copy(); square_C.apply_matrix(C)
        group_C = VGroup(e1_C, e2_C, square_C).next_to(square_BA, DOWN, buff=0.6)
        self.play(FadeIn(group_C))

        eq_label = MathTex("A(B(x)) = Cx").scale(0.9).next_to(matC, RIGHT, buff=0.4)
        self.play(Write(eq_label))
        self.wait(1.0)

        # === Entrywise computation visualization ===
        symA = Matrix([["a","b"],["c","d"]]).scale(0.8).to_edge(LEFT)
        symB = Matrix([["e","f"],["g","h"]]).scale(0.8).to_edge(RIGHT)
        symC = Matrix([["ae+bg","af+bh"],["ce+dg","cf+dh"]]).scale(0.8).next_to(symA, DOWN, buff=0.6)
        self.play(Write(symA), Write(symB))
        self.wait(0.5)

        entriesA = symA.get_entries()
        entriesB = symB.get_entries()
        row0_box = SurroundingRectangle(VGroup(entriesA[0], entriesA[1]), color=RED)
        col0_box = SurroundingRectangle(VGroup(entriesB[0], entriesB[2]), color=YELLOW)
        self.play(Create(row0_box), Create(col0_box))
        self.wait(0.4)

        expr = MathTex(
            f"{A[0,0]:.2f}", "\\cdot", f"{B[0,0]:.2f}", "+",
            f"{A[0,1]:.2f}", "\\cdot", f"{B[1,0]:.2f}", "=", f"{C[0,0]:.2f}"
        ).next_to(symC, RIGHT, buff=0.4)
        self.play(Write(expr))
        self.wait(1.0)

        self.play(FadeOut(VGroup(symA, symB, symC, row0_box, col0_box, expr)))

        # === Eigenvectors of A (if real) ===
        eigvals, eigvecs = np.linalg.eig(A)
        tol = 1e-6
        for i in range(len(eigvals)):
            vec = eigvecs[:, i]
            if not np.allclose(vec.imag, 0, atol=tol):
                continue
            v = vec.real / np.linalg.norm(vec.real)
            arrow = Arrow(plane.c2p(0,0), plane.c2p(v[0], v[1]), buff=0).set_color(PURPLE)
            scaled = Arrow(plane.c2p(0,0), plane.c2p(eigvals[i].real*v[0], eigvals[i].real*v[1]), buff=0)
            label = MathTex(f"\\lambda_{i}={eigvals[i]:.2f}").next_to(scaled.get_end(), UR)
            self.play(GrowArrow(arrow), FadeIn(label))
            self.play(Transform(arrow.copy(), scaled), run_time=1.0)
            self.wait(0.4)

        self.wait(1.2)

%manim -qk -v warning MatrixTransformScene